In [1]:
import hashlib
from base64 import b64encode, b64decode
import pandas as pd
import qrcode as qr
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad


def encrypt_with_AES_CBC(secretKey, data, iv):
    try:
        keyBytes = hashlib.sha256(secretKey.encode('utf-8')).digest()
        cipher = AES.new(keyBytes, AES.MODE_CBC, iv.encode('utf-8'))
        cipherText = cipher.encrypt(pad(data.encode('utf-8'), AES.block_size))
        return b64encode(cipherText).decode('utf-8')
    except Exception as e:
        print(str(e))
        return None


def decrypt_with_AES_CBC(secretKey, data, iv):
    try:
        keyBytes = hashlib.sha256(secretKey.encode('utf-8')).digest()
        cipher = AES.new(keyBytes, AES.MODE_CBC, iv.encode('utf-8'))
        plainText = unpad(cipher.decrypt(b64decode(data.encode('utf-8'))), AES.block_size)
        return plainText.decode('utf-8').strip()
    except Exception as e:
        print(str(e))
        return None

In [2]:
file_path = r"D:\TA Materials\الفرق 2024\All.xlsx"
sheet_name = "Sheet1"
students = pd.read_excel(file_path, header=None, sheet_name=sheet_name)

In [3]:
students.head()

,0,1,2
0,ابراهيم احمد محمود محمد حسين,مستجد,الفرقة الاولى - عام
1,ابراهيم حسين عبدالعاطي احمد,مستجد,الفرقة الاولى - عام
2,ابراهيم دسوقي حجازي عبدالمعطي,باقي,الفرقة الاولى - عام
3,ابراهيم رضا ابراهيم عبدالفتاح,باقي,الفرقة الاولى - عام
4,ابراهيم عاطف السيد عبدالرحمن,مستجد,الفرقة الاولى - عام


In [7]:
# create a folder for each group
import os

for group in students[2].unique():
    os.makedirs(r"D:\QR Codes\\" + str(group), exist_ok=True)

|### Add new student to the dataframe

### Create Qr code for the new students

In [8]:
students.head()

,0,1,2
0,ابراهيم احمد محمود محمد حسين,مستجد,الفرقة الاولى - عام
1,ابراهيم حسين عبدالعاطي احمد,مستجد,الفرقة الاولى - عام
2,ابراهيم دسوقي حجازي عبدالمعطي,باقي,الفرقة الاولى - عام
3,ابراهيم رضا ابراهيم عبدالفتاح,باقي,الفرقة الاولى - عام
4,ابراهيم عاطف السيد عبدالرحمن,مستجد,الفرقة الاولى - عام


In [11]:
# create a qr code of the name of the student and his roll number and save it in the folder
for i in range(students.shape[0]):
    qr_data = str(students.iloc[i, 0]) + "\n" + str(i + 2)
    secretKey = "Secret Key"  # replace with your secret key
    iv = "##############" # replace with your iv
    qr_data = encrypt_with_AES_CBC(secretKey, qr_data, iv)
    qr.make(qr_data).save(r"D:\QR Codes\QR" + str(i) + ".png")
    qr.make(qr_data).save(r"D:\QR Codes\\"+f"{students.iloc[i,2]}\\" + str(students.iloc[i,0])+".png")

    # students["QR Code"] = "QR" + str(i) + ".png"

In [12]:
writer = pd.ExcelWriter('all.xlsx', engine='xlsxwriter')
students.to_excel(writer, sheet_name='Sheet1', index=False)

In [13]:
from PIL import Image

workbook = writer.book
my_format = workbook.add_format()
my_format.set_align('vcenter')

worksheet = writer.sheets['Sheet1']
worksheet.set_default_row(44)

default_column_width = 8.43

for i in range(students.shape[0]):
    worksheet.write('B' + str(i + 2),students.iloc[i, 2], my_format)
    worksheet.write('C' + str(i + 2),students.iloc[i, 0], my_format)
    image_path = r"D:\QR Codes\QR" + str(i) + ".png"
    # Get the dimensions of the image
    with Image.open(image_path) as img:
        image_width, image_height = img.size

    # Calculate the scale factors
    x_scale = 55 / image_width
    y_scale = 55 / image_height

    # Calculate the scaled image width and height
    scaled_image_width = image_width * x_scale
    scaled_image_height = image_height * y_scale

    # Calculate the offsets
    x_offset = int((60 - scaled_image_width) / 2)
    y_offset = int((60 - scaled_image_height) / 2)

    # Insert the image
    worksheet.set_column('A:A', None, my_format)
    worksheet.insert_image('A' + str(i + 2), r"D:\QR Codes\QR" + str(i) + ".png",
                           {'x_scale': x_scale, 'y_scale': y_scale, 'x_offset': x_offset, 'y_offset': y_offset})


In [14]:
writer.save()

<img src="img.png"/>